In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

print(torch.__version__)

1.7.0+cu101


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# Load MNIST dataset
mnist_train = torchvision.datasets.MNIST(
    root = 'drive/MyDrive/pytorch/practice/MNIST_data/',
    train=True,
    transform = transforms.ToTensor(),
    download=True
)

mnist_test = torchvision.datasets.MNIST(
    root = 'drive/MyDrive/pytorch/practice/MNIST_data/',
    train=False,
    transform = transforms.ToTensor(),
    download=True
)

In [5]:
# Dataset loader
data_loader = torch.utils.data.DataLoader(mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True)

In [6]:
# Check data
for data in data_loader:
  print("Data: ", data)
  break

Data:  [tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0., 

In [7]:
# Check data
for data in test_loader:
  print("Data: ", data)
  break

Data:  [tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0., 

In [13]:
# Create model
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.drop1 = nn.Dropout2d(0.25)
    self.drop2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(1600, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):

    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.drop1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.drop2(x)
    x = self.fc2(x)

    output = F.softmax(x, dim=1)
    return output

In [14]:
# test model
random_data = torch.rand((1, 1, 28, 28))

model = Net().to(device)

In [15]:
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (drop1): Dropout2d(p=0.25, inplace=False)
  (drop2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [16]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# Train model
total_batch = len(data_loader)
model.train()
print('Learning started...')
for epoch in range(training_epochs):
  losses = 0

  for images, labels in data_loader:
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    losses += loss / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, losses))
  
print('Learning Finished!')

Learning started...
[Epoch:    1] cost = 1.61518574
[Epoch:    2] cost = 1.51301372
[Epoch:    3] cost = 1.5015974
[Epoch:    4] cost = 1.49631751
[Epoch:    5] cost = 1.49381638
[Epoch:    6] cost = 1.48951745
[Epoch:    7] cost = 1.48827887
[Epoch:    8] cost = 1.48763633
[Epoch:    9] cost = 1.48571932
[Epoch:   10] cost = 1.48470128
[Epoch:   11] cost = 1.48373103
[Epoch:   12] cost = 1.48225105
[Epoch:   13] cost = 1.48174238
[Epoch:   14] cost = 1.48063421
[Epoch:   15] cost = 1.47939765
Learning Finished!


In [18]:
# Test model and check accuray
with torch.no_grad():
  model.eval()
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item()*100)

Accuracy:  98.6799955368042


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [19]:
# save model
path = 'drive/MyDrive/pytorch/practice/mnist_model2.pt'
torch.save(model.state_dict(), path)